## [Effective Python Testing With Pytest](https://realpython.com/pytest-python-testing/)

#### What Makes pytest So Useful?
- Most functional tests follow the Arrange-Act-Assert model:
    1. Arrange, or set up, the conditions for the test
    2. Act by calling some function or method
    3. Assert that some end condition is true

- `pytest` simplifies this workflow by allowing you to use normal functions and Python’s assert keyword directly:
```python
def test_always_passes():
    assert True

def test_always_fails():
    assert False
```
- You don’t need any imports or classes. All you need to do is include a function with the `test_` prefix. Because you can use the `assert` keyword, you don’t need to learn or remember all the different `self.assert*` methods in `unittest`, either. If you can write an expression that you expect to evaluate to True, and then pytest will test it for you.
- `pytest` eliminates a lot of boilerplate and also provides a much more detailed and easy-to-read output.

##### Nicer Output
- You can run your test suite using the `pytest` command from the top-level folder of your project:
```bash
pytest
```
- `pytest` tests `unittest` as well if they follow the naming convention `test_`.
- Output
    - `.` means test passed.
    - `F` means test failed.
    - `E` means test raised an unexpected exception.

##### Easier to Manage State and Dependencies
- Your tests will often depend on types of data or test doubles that mock objects your code is likely to encounter, such as dictionaries or JSON files.
- With `unittest`, you might extract these dependencies into `.setUp()` and `.tearDown()` methods so that each test in the class can make use of them. Using these special methods is fine, but as your test classes get larger, you may inadvertently make the test’s dependence entirely **implicit**. In other words, by looking at one of the many tests in isolation, you may not immediately see that it depends on something else. Over time, implicit dependencies can lead to a complex tangle of code that you have to unwind to make sense of your tests. Tests should help to make your code more understandable.
- `pytest` takes a different approach. It leads you toward **explicit** dependency declarations that are still reusable thanks to the availability of `fixtures`. `pytest` `fixtures` are functions that can create data, test doubles, or initialize system state for the test suite. Any test that wants to use a fixture must explicitly use this fixture function as an argument to the test function, so dependencies are always stated up front:
```python
import pytest

@pytest.fixture
def example_fixture():
    return 1

def test_with_fixture(example_fixture):
    assert example_fixture == 1
```

- Fixtures can also make use of other fixtures, again by declaring them explicitly as dependencies. That means that, over time, your fixtures can become bulky and modular. (explained later in detail)

##### Easy to Filter Tests
- As your test suite grows, you may find that you want to run just a few tests on a feature and save the full suite for later. `pytest` provides a few ways of doing this:
    - **Name-based filtering**: You can limit pytest to running only those tests whose fully qualified names match a particular expression. You can do this with the `-k` parameter.
    - **Directory scoping**: By default, `pytest` will run only those tests that are in or under the current directory.
    - **Test categorization**: `pytest` can include or exclude tests from particular categories that you define. You can do this with the `-m` parameter.
- `pytest` enables you to create marks, or custom labels, for any test you like. A test may have multiple labels, and you can use them for granular control over which tests to run. (explained later in detail)

##### Allows Test Parametrization
- When you’re testing functions that process data or perform generic transformations, you’ll find yourself writing many similar tests. They may differ only in the input or output of the code being tested. This requires duplicating test code, and doing so can sometimes obscure the behavior that you’re trying to test.
- `pytest` offers solution in which each test can pass or fail independently (which is not the case in `unittest`).

#### Fixtures: Managing State and Dependencies
##### How to Use Fixtures at Scale
- In `pytest`, fixtures are **modular**. Being modular means that fixtures can be imported, can import other modules, and they can depend on and import other fixtures.
- If you want to make a fixture available for your whole project without having to import it, a special configuration module called *conftest.py* will allow you to do that. `pytest` looks for a *conftest.py* module in each directory. If you add your general-purpose fixtures to the *conftest.py* module, then you’ll be able to use that fixture throughout the module’s parent directory and in any subdirectories without having to import it.
- Another interesting use case for fixtures and conftest.py is in guarding access to resources. Imagine that you’ve written a test suite for code that deals with API calls. You want to ensure that the test suite doesn’t make any real network calls even if someone accidentally writes a test that does so.
- `pytest` provides a *monkeypatch* fixture to replace values and behaviors, which you can use to great effect:
```python
import pytest
import requests

@pytest.fixture(autouse=True)
def disable_network_calls(monkeypatch):
    def stunted_get():
        raise RuntimeError("Network access not allowed during testing!")
    monkeypatch.setattr(requests, "get", lambda *args, **kwargs: stunted_get())
```
- By placing `disable_network_calls()` in *conftest.py* and adding the `autouse=True` option, you ensure that network calls will be disabled in every test across the suite. Any test that executes code calling `requests.get()` will raise a `RuntimeError` indicating that an unexpected network call would have occurred.

#### Marks: Categorizing Tests
- `pytest` enables you to define categories for your tests and provides options for including or excluding categories when you run your suite. You can mark a test with any number of categories.
- Because you can give your marks any name you want, it can be easy to mistype or misremember the name of a mark. `pytest` will warn you about marks that it doesn’t recognize in the test output. You can use the `--strict-markers` flag to the pytest command to ensure that all marks in your tests are registered in your pytest configuration file, `pytest.ini`. It’ll prevent you from running your tests until you register any unknown marks.
-  If you’d like to run only those tests that require database access (for example), then you can use `pytest -m database_access`. To run all tests except those that require database access, you can use `pytest -m "not database_access"`. You can even use an autouse fixture to limit database access to those tests marked with database_access.
- The `pytest-django` plugin, for instance, provides a `django_db` mark. Any tests without this mark that try to access the database will fail. The first test that tries to access the database will trigger the creation of Django’s test database.

- `pytest` provides a few marks out of the box:
    - *skip* skips a test unconditionally.
    - *skipif* skips a test if the expression passed to it evaluates to True.
    - *xfail* indicates that a test is expected to fail, so if the test does fail, the overall suite can still result in a passing status.
    - *parametrize* creates multiple variants of a test with different values as arguments. (later)
- You can see a list of all the marks that pytest knows about by running `pytest --markers`.

#### Parametrization: Combining Tests
```python
@pytest.mark.parametrize("palindrome", [
    "",
    "a",
    "Bob",
    "Never odd or even",
    "Do geese see God?",
])
def test_is_palindrome(palindrome):
    assert is_palindrome(palindrome)

@pytest.mark.parametrize("non_palindrome", [
    "abc",
    "abab",
])
def test_is_palindrome_not_palindrome(non_palindrome):
    assert not is_palindrome(non_palindrome)
```

- The first argument to parametrize() is a comma-delimited string of parameter names. The second argument is a list of either tuples or single values that represent the parameter value(s). You could take your parametrization a step further to combine all your tests into one:
```python
# not advised as it combine two different assert conditions
@pytest.mark.parametrize("maybe_palindrome, expected_result", [
    ("", True),
    ("a", True),
    ("Bob", True),
    ("Never odd or even", True),
    ("Do geese see God?", True),
    ("abc", False),
    ("abab", False),
])
def test_is_palindrome(maybe_palindrome, expected_result):
    assert is_palindrome(maybe_palindrome) == expected_result
```

#### Useful pytest Plugins
##### pytest-randomly
- pytest-randomly forces your tests to run in a random order. This is a great way to uncover tests that depend on running in a specific order, which means they have a stateful dependency on some other test.
##### pytest-cov
- If you want to measure how well your tests cover your implementation code, then you can use the coverage package.